### 1.Read and show video stream,Capture images
### 2.Detect facing using haarcascade and show bounding box(haarcascade)
### 3.Flatten the largest face image(gray scale) as to save memory and save in numpy array
### 4.Repeat the above for multiple face to generate training data
### 5. If there are multiple faces in frame/screen we would take largest face

##### f=x,y,w,h   
if there are more than one faces then we need to select the area with largest face so sorting is required which will be do sort all the faces over Area calulated WxH so F[2]*F[3] (key)

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [ ]:

#Init Camera
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")

skip = 0
face_data = [] #array
dataset_path = './data/'
file_name = input("Enter the name of person whose face we are scanning")
while True:
	ret,frame = cap.read()

	if ret==False:
		continue

	gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
	

	faces = face_cascade.detectMultiScale(frame,1.3,5) # faces will list of tuples/faces as faces is tupple-[(10,30,20,40),(12,14,15,67)] represeing [(x,y,w,h)]
	#if len(faces)==0:
		#continue
	#print(faces) # it will print matrix
	
	faces = sorted(faces,key=lambda f:f[2]*f[3])

	#Pick the largest face(because it is the largest face according to area f[2]*f[3])

	for face in faces[-1:]: # -1 means last face : till the end , so only 1 face will be captured which is last one
		x,y,w,h = face
		cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2) # drawing rectangle on the frame

		#Extract (Crop out the required face) : region of Interest
		offset = 10 # means padding of 10 pixels to boundary
		face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]# frame[y,x]  in frame first axis is y axis and second axis is x axis.
		face_section = cv2.resize(face_section,(100,100)) # resizing it into 100x100 Pixels
        #store every 10th face
		skip += 1
		if skip%10==0:
			face_data.append(face_section)
			print(len(face_data))#count of the face


	cv2.imshow("Frame",frame)
	cv2.imshow("Face Section",face_section)

	key_pressed = cv2.waitKey(1) & 0xFF
	if key_pressed == ord('q'):
		break

#convert our face list into numpy array
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0],-1))# no. of rows= no. of faces and columns should be fighured out automatically
print(face_data.shape)

# Save this data into file system
np.save(dataset_path+file_name+'.npy',face_data)#data/a.npy or data/b.npy
print("Data Successfully save at "+dataset_path+file_name+'.npy') # 2nd portion is string

cap.release()
cv2.destroyAllWindows()

# the shape s 14,30000. 30000 because we took rgb channel because 3 is multiplied for RGB frame
# if you took gray_frame then dimension would be 14,10000
